In [1]:
import sys
sys.path.append('../')
from python_modules.comparisonframe import ComparisonFrame

/home/kyriosskia/miniconda3/envs/testenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Usage examples

The examples contain: 
1. Creating validation set and saving it to be reused
2. Comparing newly generated data with expected results 
3. Calculating aggregate comparison scores
4. Recording test statuses
5. Reseting statuses, flushing records and comparison results

### 1. Creating validation set

### 1.1 Initialize comparison class

In [2]:
comparer = ComparisonFrame(
    # optionally
    ## mocker default parameters
    mocker_params = {
        'file_path' : "./comparisonframe_storage",
         'persist' : True},

    ## scores to calculate
    compare_scores = ['char_count_diff',
                        'word_count_diff',
                        'line_count_diff',
                        'punctuation_diff',
                        'semantic_similarity']
)

/home/kyriosskia/miniconda3/envs/testenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### 1.2 Recording queries and expected responses (validation set)

In [3]:
comparer.record_queries(
    queries = ["Black metal", 
               "Tribulation"],
    expected_texts = ["Black metal is an extreme subgenre of heavy metal music.",
    "Tribulation are a Swedish heavy metal band from Arvika that formed in 2005."],
    metadata = {'name' : 'metal_bands'})

### 2. Comparing newly generated data with expected results 

#### 2.1 Initialize new comparison class

In [4]:
comparer = ComparisonFrame()

### 2.2 Show validation set

In [5]:
untested_queries = comparer.get_all_queries(
    ## optional
    metadata_filters={'name' : 'metal_bands'})
print(untested_queries)

['Black metal', 'Tribulation']


In [6]:
comparer.get_all_records()

[{'query': 'Black metal',
  'record_id': '0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8bd7b663c92f2f16e87',
  'expected_text': 'Black metal is an extreme subgenre of heavy metal music.'},
 {'query': 'Tribulation',
  'record_id': 'eecd9c2a5b25ee6053891b894157fa30372ed694763385e1ada1dc9ad8e41625',
  'expected_text': 'Tribulation are a Swedish heavy metal band from Arvika that formed in 2005.'}]

In [7]:
comparer.get_all_records_df()

,query,record_id,expected_text
0,Black metal,0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8...,Black metal is an extreme subgenre of heavy me...
1,Tribulation,eecd9c2a5b25ee6053891b894157fa30372ed694763385...,Tribulation are a Swedish heavy metal band fro...


#### 2.3 Insert newly generated with records

In [8]:
valid_answer_query_1 = "Black metal is an extreme subgenre of heavy metal music."
very_similar_answer_query_1 = "Black metal is a subgenre of heavy metal music."
unexpected_answer_query_1 = "Black metals are beautiful and are often used in jewelry design."

In [9]:
comparer.record_runs(queries = ["Black metal"],
                     provided_texts = [valid_answer_query_1,
                                      very_similar_answer_query_1,
                                      unexpected_answer_query_1],
                    metadata={'desc' : 'definitions'})

In [10]:
df = comparer.get_all_runs_df()
df

,query,run_id,timestamp,provided_text
0,Black metal,84ba591f22f9a85d02b819cc975406050179159c153462...,2024-08-21 23:06:21,Black metal is an extreme subgenre of heavy me...
1,Black metal,f8decbaf2684cb3910a1b6dd585e33e15f7affff173ab3...,2024-08-21 23:06:21,Black metal is a subgenre of heavy metal music.
2,Black metal,dc09a47365766f8d414e71081d5960930e8f323d76abed...,2024-08-21 23:06:21,Black metals are beautiful and are often used ...


### 2.4 Comparing runs with records

In [11]:
comparer.compare_runs_with_records()

In [12]:
comparer.get_all_run_scores_df()

,query,run_id,timestamp,provided_text,record_id,char_count_diff,word_count_diff,line_count_diff,punctuation_diff,semantic_similarity,comparison_id
0,Black metal,84ba591f22f9a85d02b819cc975406050179159c153462...,2024-08-21 23:06:21,Black metal is an extreme subgenre of heavy me...,0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8...,0,0,0,0,1.000000,76527d9ba703d734ba3362e10706f6734028e648d8362b...
1,Black metal,f8decbaf2684cb3910a1b6dd585e33e15f7affff173ab3...,2024-08-21 23:06:21,Black metal is a subgenre of heavy metal music.,0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8...,9,1,0,0,0.985985,ec33ef2dc6c3125c5606d110ac301435cfe350c055da93...
2,Black metal,dc09a47365766f8d414e71081d5960930e8f323d76abed...,2024-08-21 23:06:21,Black metals are beautiful and are often used ...,0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8...,8,1,0,0,0.494053,aafc4e6ef286d89086d04d48ca33edd39f0aacfb07d8bc...


### 3 Calculating aggregate comparison scores

In [13]:
comparer.calculate_aggr_scores()

In [23]:
comparer.get_all_aggr_scores_df()

,timestamp,comparison_id,query,min_char_count_diff,min_word_count_diff,min_line_count_diff,min_punctuation_diff,min_semantic_similarity,p25_char_count_diff,p25_word_count_diff,...,p75_word_count_diff,p75_line_count_diff,p75_punctuation_diff,p75_semantic_similarity,max_char_count_diff,max_word_count_diff,max_line_count_diff,max_punctuation_diff,max_semantic_similarity,record_status_id
0,2024-08-21 23:06:21,[76527d9ba703d734ba3362e10706f6734028e648d8362...,Black metal,0.0,0.0,0.0,0.0,0.494053,4.0,0.5,...,1.0,0.0,0.0,0.992993,9.0,1.0,0.0,0.0,1.0,79da5e839e21568134a6185f0a8164879526b1a8102b53...


### 4. Recording test statuses

In [3]:
comparer.calculate_test_statuses(test_query = "p75_semantic_similarity > 0")


In [3]:
comparer.get_test_statuses()

[{'timestamp': '2024-08-21 23:50:32',
  'record_id': '0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8bd7b663c92f2f16e87',
  'record_status_id': '79da5e839e21568134a6185f0a8164879526b1a8102b5334711f5c470a872db8',
  'query': 'Black metal',
  'valid': True}]

In [4]:
comparer.get_test_statuses_df()

,timestamp,record_id,record_status_id,query,valid
0,2024-08-21 23:50:32,0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8...,79da5e839e21568134a6185f0a8164879526b1a8102b53...,Black metal,True


### 5. Reseting statuses, flushing records and comparison results